In [14]:
from azureml.core import Workspace, Experiment, Run, Model

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to work with', ws.name)

Ready to work with dsba6190ml


In [15]:
## Get experiment and run of interest
experiment = Experiment(workspace=ws, name='instructor_diabetes')
run = Run(experiment, run_id = "instructor_diabetes_1679526317_8dca7acb")

In [16]:
## Register the model
print('Registering model...')
run.register_model(model_path='outputs/diabetes_rfcfinal_model.pkl', model_name='inst_diabetes_model',
                   tags={'Training context':'Inline Training'})

## Get the registered model
model = ws.models['inst_diabetes_model']

print('Model registered.')

Registering model...
Model registered.


## Deploy a model as a web service

In [17]:
import os

## Create a folder for the deployment files
deployment_folder = './diabetes_service'
os.makedirs(deployment_folder, exist_ok=True)
print(deployment_folder, 'folder created.')

## Set path for scoring script
script_file = 'score_diabetes.py'
script_path = os.path.join(deployment_folder,script_file)

./diabetes_service folder created.


Now you need an entry script that the service will use to score new data.

In [18]:
%%writefile $script_path
import json
import joblib
import numpy as np
import os
import pandas as pd

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'diabetes_rfcfinal_model.pkl')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = json.loads(raw_data)['data']

    df = pd.DataFrame(data,
                      columns = [
                          "PatientID",
                          "Pregnancies",
                          "PlasmaGlucose",
                          "DiastolicBloodPressure",
                          "TricepsThickness",
                          "SerumInsulin",
                          "BMI",
                          "DiabetesPedigree",
                          "Age"
                      ] 
                     )
    
    # Get a prediction from the model
    predictions = model.predict(df)
    
    # print the data and predictions (so they'll be logged!)
    log_text = 'Data:' + str(data) + ' - Predictions:' + str(predictions)
    print(log_text)
    
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

Overwriting ./diabetes_service/score_diabetes.py


Now you can deploy the service (in this case, as an Azure Container Instance (ACI).

> **Note**: This can take a few minutes - wait until the state is shown as **Healthy**.

In [22]:
from azureml.core.conda_dependencies import CondaDependencies 

# Add the dependencies for our model (AzureML defaults is already included)
diabetes_env = CondaDependencies()
diabetes_env.add_conda_package('scikit-learn')
diabetes_env.add_conda_package('pandas')

# Save the environment config as a .yml file
env_file = os.path.join(deployment_folder, "diabetes_env.yml")
with open(env_file, "w") as f:
    f.write(diabetes_env.serialize_to_string())
print("Saved dependency info in", env_file)

# Print the .yml file
with open(env_file,"r") as f:
    print(f.read())

Saved dependency info in ./diabetes_service/diabetes_env.yml
# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.8 and later.
- python=3.8.13

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults

- scikit-learn
- pandas
channels:
- anaconda
- conda-forge



In [24]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   entry_script=script_path,
                                   conda_file=env_file)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name = "inst-diabetes-service"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

/tmp/ipykernel_11697/3525972235.py:13: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-03-23 02:44:20+00:00 Creating Container Registry if not exists.
2023-03-23 02:44:20+00:00 Registering the environment.
2023-03-23 02:44:21+00:00 Use the existing image.
2023-03-23 02:44:21+00:00 Generating deployment configuration.
2023-03-23 02:44:22+00:00 Submitting deployment to compute.
2023-03-23 02:44:27+00:00 Checking the status of deployment inst-diabetes-service..
2023-03-23 02:46:21+00:00 Checking the status of inference endpoint inst-diabetes-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


## Use the web service

With the service deployed, now you can consume it from a client application.

First, determine the URL to which these applications must submit their requests.

In [26]:
endpoint = service.scoring_uri
print(endpoint)

http://fcfa5230-46e2-4320-bf1d-0c205e8394b3.eastus.azurecontainer.io/score


Now that you know the endpoint URI, an application can simply make an HTTP request, sending the patient data in JSON (or binary) format, and receive back the predicted class(es).

> **Tip**: If an error occurs because the service endpoint isn't ready. Wait a few seconds and try again!

In [27]:
import requests
import json

# Create new data for inferencing
# x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
#          [0,148,58,11,179,39.19207553,0.160829008,45]]

x_new = {"data": [
    [1, 2,180,74,24,21,23.9091702,1.488172308,22],
    [2, 0,148,58,11,179,39.19207553,0.160829008,45],
    [1218879, 1,124,82,42,266,34.98577243,0.083335019,25],
    [1940297, 2,44,81,46,146,34.53408234,0.693502171,55]
]}

# Convert the array to a serializable list in a JSON document
input_json = json.dumps(x_new)

# Set the content type
headers = { 'Content-Type':'application/json' }

# Get the predictions
predictions = requests.post(endpoint, input_json, headers = headers)
print(predictions.status_code)

if predictions.status_code == 200:
    predicted_classes = json.loads(predictions.json())
    for i in range(len(x_new['data'])):
        print ("Patient {}".format(x_new['data'][i]), predicted_classes[i] )

200
Patient [1, 2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22] diabetic
Patient [2, 0, 148, 58, 11, 179, 39.19207553, 0.160829008, 45] diabetic
Patient [1218879, 1, 124, 82, 42, 266, 34.98577243, 0.083335019, 25] diabetic
Patient [1940297, 2, 44, 81, 46, 146, 34.53408234, 0.693502171, 55] diabetic
